# 研究データ管理計画を立てる

研究データ管理計画を策定します。<br>
研究データ管理計画は、DMP(データマネジメントプラン)とデータガバナンス機能のオリジナル項目によって策定します。<br>
研究データ管理計画を策定後、それに基づいたリサーチフローとガバナンスシートが用意されます。<br>

## 1. 研究データ管理計画（DMP）を取得する
研究データ管理計画（DMP）をGRDMから取り込みます。<br>
GRDMでDMP（プロジェクトメタデータ）を登録していない場合は、GRDMに戻ってプロジェクトメタデータを作成してください。<br>
DMPが登録されている場合は、リサーチフローでそのDMを取り込みますので、GRDMに戻ってプロジェクトメタデータをCSV出力してください。<br>
CSVが手元にある場合はここでそのCSVの取込を実行します。

In [ ]:
# TODO: Implement

## 2. DGカスタマイズプロパティの設定
研究データ管理計画をDMPとデータガバナンス機能のオリジナル項目によって策定します。<br>
DMPの登録は任意ですが、データガバナンス機能のオリジナル項目はリサーチフローを提供するために登録が必須です。

In [ ]:
# 研究データ管理計画を策定する。
import os
import sys
sys.path.append('../../../../..')
from library.plan.make_research_data_management_plan import DGPlaner

DGPlaner(working_path=os.path.abspath('__file__')).generateFormScetion()

## 3. GRDMに実行結果を同期
タスクの実行結果をGRDMに同期します。

In [ ]:
# TODO: Implement

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
# サブフローメニューへアクセスするボタンを表示
import os
import sys
sys.path.append('../../../../..')
from library.plan.make_research_data_management_plan import DGPlaner
DGPlaner(working_path=os.path.abspath('__file__')).return_subflow_menu()